In [1]:
import pandas as pd
import numpy as np
import sklearn
import gc
gc.collect()
import pickle
import os 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
df=pd.read_csv("data.csv.zip",nrows=20000)

In [3]:
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [4]:
columns = ['artists','name','popularity','release_date']

In [5]:
def get_imp_features(data):
  important_features=[]
  for i in range(0,data.shape[0]):
    important_features.append(data['artists'][i]+' '+data['name'][i]+' '+str(data['popularity'][i])+' '+str(data['release_date'][i]))

  return important_features

In [6]:
df['important_features']= get_imp_features(df)
df.head(100)


,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,important_features
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.2110,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.6650,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954,"['Sergei Rachmaninoff', 'James Levine', 'Berli..."
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.3410,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.1600,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936,['Dennis Day'] Clancy Lowered the Boom 5 1921
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.1660,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.1010,-14.850,1,Gati Bali,5,1921,0.0339,110.339,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.3090,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.3810,-9.316,1,Danny Boy,3,1921,0.0354,100.109,['Frank Parker'] Danny Boy 3 1921
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.1930,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.2290,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665,['Phil Regan'] When Irish Eyes Are Smiling 2 1921
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.1730,1921,0.992,['Morton Downey'],0.380,170240,0.1170,0,3tEwdlwf8ZrxpksJorI5Tc,0.000001,7,0.1050,-13.298,1,That's How You Spell Ireland,0,1921,0.0415,86.493,['Morton Downey'] That's How You Spell Ireland...
96,0.2190,1921,0.672,['Mehmet Kemiksiz'],0.403,396356,0.3850,0,47LO5zrjx6ShMymLqp80ef,0.000339,11,0.2110,-6.507,0,Şühedâ Gövdesi Bir Baksana Dağlar Taşlar,0,1921,0.0315,123.839,['Mehmet Kemiksiz'] Şühedâ Gövdesi Bir Baksana...
97,0.9190,1921,0.959,['Morton Downey'],0.829,153827,0.3330,0,4BWXQSxvAa73YTLVLlMtGp,0.000362,5,0.0841,-11.895,1,It's the Same Old Shillelegh,0,1921,0.1030,132.606,['Morton Downey'] It's the Same Old Shillelegh...
98,0.4510,1921,0.995,['Ignacio Corsini'],0.592,168053,0.0322,0,4Rv29R4ydhHN9tYPfrZxTz,0.057300,5,0.2970,-25.844,0,China Hereje - Remasterizado,0,1921-03-20,0.2090,69.958,['Ignacio Corsini'] China Hereje - Remasteriza...


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [8]:
vector = cv.fit_transform(df['important_features']).toarray()
print(vector)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [9]:
vector.shape

(20000, 5000)

In [10]:
#cm = CountVectorizer().fit_transform(df['important_features'])

In [11]:
cs =cosine_similarity(vector)
print(cs)

[[1.         0.12909944 0.09128709 ... 0.         0.         0.        ]
 [0.12909944 1.         0.1767767  ... 0.         0.         0.        ]
 [0.09128709 0.1767767  1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         1.         0.13483997]
 [0.         0.         0.         ... 1.         1.         0.13483997]
 [0.         0.         0.         ... 0.13483997 0.13483997 1.        ]]


In [19]:
#df[df['name'] == 'Danny Boy'].index[0]

In [20]:
def recommend(song):
    index = df[df['name'] == song].index[0]
    distances = sorted(list(enumerate(cs[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(df.iloc[i[0]])
        print("****************************************************************")
        
        
        
        

In [21]:
k=recommend("Danny Boy")

valence                                                       0.197
year                                                           1921
acousticness                                                  0.975
artists                                       ['Christopher Lynch']
danceability                                                  0.418
duration_ms                                                  151840
energy                                                       0.0904
explicit                                                          0
id                                           2ksYHAsGBLxAupsni2QeNw
instrumentalness                                           0.000003
key                                                               8
liveness                                                      0.112
loudness                                                    -13.871
mode                                                              1
name                                            

In [15]:
print(df.iloc[27])

valence                                                  0.723
year                                                      1921
acousticness                                             0.388
artists                                    ['Mehmet Kemiksiz']
danceability                                             0.685
duration_ms                                             155063
energy                                                   0.698
explicit                                                     0
id                                      0osXBirvQzPRfKSUDzHPCv
instrumentalness                                      0.000002
key                                                          4
liveness                                                0.0768
loudness                                                -8.184
mode                                                         0
name                                             Korkma Sönmez
popularity                                             